# 輸入小考名稱

In [1]:
# @title
# ====== 設定目標小考名稱 ======
print('\n請輸入小考名稱，例如：204\n')
target_exam_name = input()


請輸入小考名稱，例如：204



KeyboardInterrupt: Interrupted by user

# 上傳答案檔案

檔案格式欄位應為

|小考名稱|題型數量|配分|選擇1答案（可不寫）|選擇2答案（可不寫）|...|
|-|-|-|-|-|-|
|204|2,3,3,1,3,2,3,4,4,4,2,3,5,3,2,2,2|6,6,4,2,6|

題型數量欄位填入：單選題數,多選題數,第1題選填題有幾格,第2題選填題有幾格,...

配分欄位填入：單選題每題分數,多選題全對分數,多選題答錯1題分數,多選題答錯2題分數,選填題每題分數

如果不確定格式的話，可以直接到文末找：答案範本格式輸出。

若題數設定有誤會跑出警語！

In [ ]:
# @title
import pandas as pd
import sys
from google.colab import files
import io
import os



# ====== 題型數量設定 ======
#單選 num_single_choice
#多選 num_multiple_choice
#選填 fill_choice_num_cells
# 單選題數,多選題數,第1題選填題有幾格,第2題選填題有幾格,...
num_single_choice, num_multiple_choice, *fill_choice_num_cells = 0, 0, []

# # ====== 配分設定  ======
# 選擇題配分 single_choice_score
# 多選題全對 multiple_choice_score_full
# 多選題錯一 multiple_choice_score_one_wrong
# 多選題錯二 multiple_choice_score_two_wrong
# 多選題錯三 multiple_choice_score_three_wrong
# 多選題沒寫 multiple_choice_score_empty
# 填充題分數 fill_choice_score_full

single_choice_score = 0
multiple_choice_score_full = 0
multiple_choice_score_one_wrong = 0
multiple_choice_score_two_wrong = 0
multiple_choice_score_three_wrong = 0
multiple_choice_score_empty = 0
fill_choice_score_full = 0
fill_choice_score_wrong = 0


# ====== 上傳答案檔案 ======
print("\n請上傳答案檔案 (例如：答案.xlsx)。")
uploaded_answer = files.upload()

answer_file = None
answer_file_name = None

if uploaded_answer:
    answer_file_name = list(uploaded_answer.keys())[0]
    try:
        answer_file_content = uploaded_answer[answer_file_name]
        answer_file = pd.read_excel(io.BytesIO(answer_file_content))
        print(f"答案檔案 '{answer_file_name}' 上傳成功。")
    except Exception as e:
        print(f"錯誤：讀取答案檔案 '{answer_file_name}' 失敗：{e}")
        sys.exit()
else:
    print("錯誤：未上傳答案檔案。")
    sys.exit()



# ====== 查找目標小考的答案 ======
correct_answers = {}
found_exam = False

for index, row in answer_file.iterrows():
    exam_name = str(row.iloc[0]).strip()
    if exam_name == target_exam_name:
        found_exam = True
        actual_columns = len(row)
        # 從答案檔案第1欄抓題型數量
        num_single_choice, num_multiple_choice, *fill_choice_num_cells =(
             [int(ele) for ele in str(row.iloc[1]).strip().split(',')]
        )
        # 從答案檔案第2欄抓配分
        single_choice_score, multiple_choice_score_full,\
         multiple_choice_score_one_wrong, multiple_choice_score_two_wrong,\
          fill_choice_score_full = (
              [int(ele) for ele in str(row.iloc[2]).strip().split(',')]
          )

        total_fill_cells = sum(fill_choice_num_cells) #選填題總數
        expected_columns = 3 + num_single_choice + num_multiple_choice + total_fill_cells # 從第3欄開始才是答案

        if actual_columns < expected_columns:
            print(f"警告：目標小考 '{target_exam_name}' 的答案欄位數量不足。")
            print(f"預期至少要有 {expected_columns} 個欄位 (包含小考名稱)，實際只有 {actual_columns} 個欄位。")
            print("請檢查您的答案檔案和題型數量設定。")
            sys.exit()

        # 讀取答案
        for i in range(num_single_choice):
            question_number = i + 1
            answer = row.iloc[i + 3]
            if pd.notna(answer):
                correct_answers[question_number] = str(answer).strip()

        for i in range(num_multiple_choice):
            question_number = num_single_choice + i + 1
            answer = row.iloc[num_single_choice + i + 3]
            if pd.notna(answer):
                correct_answers[question_number] = str(answer).strip()

        start_index = 3 + num_single_choice + num_multiple_choice
        fill_answers_count = 0
        for i, answer in enumerate(row.iloc[start_index:]):
            question_number = num_single_choice + num_multiple_choice + i + 1
            if pd.notna(answer):
                correct_answers[question_number] = str(answer).strip()
                fill_answers_count += 1
        # 檢查讀取到的答案數量
        expected_answer_count = num_single_choice + num_multiple_choice + total_fill_cells
        actual_answer_count = len(correct_answers)
        if actual_answer_count != expected_answer_count:
            print(f"警告：目標小考 '{target_exam_name}' 的答案數量不符預期。")
            print(f"預期答案數量：單選 {num_single_choice} + 多選 {num_multiple_choice} + 選填 {total_fill_cells} = {expected_answer_count}，實際讀取到 {actual_answer_count} 個答案。")
            print("請檢查您的題型數量設定或答案檔案。")
            sys.exit()

        # 檢查選填題格數設定與答案數量
        if fill_answers_count != total_fill_cells:
            print(f"警告：選填題的格數設定與答案檔案中的答案數量不符。")
            print(f"設定的總格數：{total_fill_cells}，答案檔案中讀取到的選填題答案數量：{fill_answers_count}。")
            print("請檢查您的選填題格數設定 (`fill_choice_num_cells`) 或答案檔案。")
            sys.exit()

        break # 找到目標小考後跳出迴圈

if not found_exam:
    print(f"錯誤：找不到名為 '{target_exam_name}' 的小考答案在 '{answer_file_name}' 中。")
    sys.exit()

# print(correct_answers)
print("答案已成功讀取。")

# 上傳學生作答情況


In [ ]:
# @title
# ====== 上傳學生作答檔案 ======
print("\n請上傳學生的作答檔案 (例如：101.xlsx, 102.xlsx, 103.xlsx)。")
uploaded_students = files.upload()

# 主程式
對完答案得到分數後送出excel檔

In [ ]:
# @title
import pandas as pd
import io
import os


# ====== 單選題計算 ======
def calculate_single_choice_score(student_row, correct_answers, start_col_idx, num_questions, score_per_question):
    score = 0
    for i in range(num_questions):
        student_answer = student_row.iloc[start_col_idx + i]
        correct_answer = correct_answers.get(i + 1)
        if correct_answer is not None and pd.notna(student_answer) and str(student_answer).strip() == correct_answer:
            score += score_per_question
    return score

# ====== 多選題計算 ======
def calculate_multiple_choice_score(student_row, correct_answers, start_col_idx, num_questions,
                                     full_score, one_wrong_score, two_wrong_score, three_wrong_score, empty_score):
    score = 0
    for i in range(num_questions):
        student_answer = student_row.iloc[start_col_idx + i]
        correct_answer_index = num_single_choice + i + 1
        correct_answer = correct_answers.get(correct_answer_index)
        if correct_answer is not None:
            if pd.isna(student_answer) or str(student_answer).strip() == '':
                score += empty_score
                continue

            student_choices = set(str(student_answer).replace(' ', '').split(','))
            correct_choices = set(correct_answer.replace(' ', '').split(','))

            if student_choices == correct_choices:
                score += full_score
            else:
                wrong_choices = student_choices.symmetric_difference(correct_choices)
                wrong_count = len(wrong_choices)

                if wrong_count == 1:
                    score += one_wrong_score
                elif wrong_count == 2:
                    score += two_wrong_score
                elif wrong_count >= 3:
                    score += three_wrong_score
    return score

# ====== 選填題計算 ======
def calculate_fill_choice_score(student_row, correct_answers, start_col_idx, num_cells, full_score, wrong_score, correct_answer_start_index):
    all_correct = True
    for i in range(num_cells):
        student_ans_raw = student_row.iloc[start_col_idx + i]
        correct_ans = correct_answers.get(correct_answer_start_index + i)

        student_ans_str = str(student_ans_raw).strip() if pd.notna(student_ans_raw) else ''
        correct_ans_str = str(correct_ans).strip() if correct_ans is not None else ''

        if correct_ans_str == '':
            all_correct = False
            break

        if student_ans_str != correct_ans_str:
            try:
                student_ans_float = float(student_ans_str)
                correct_ans_float = float(correct_ans_str)
                if student_ans_float != correct_ans_float:
                    all_correct = False
                    break
            except ValueError:
                all_correct = False
                break

    if all_correct:
        return full_score
    else:
        return wrong_score

# ====== 全部學生計算 ======
def calculate_scores(student_answers_sorted, correct_answers, num_single_choice, num_multiple_choice, fill_choice_num_cells,
                     single_choice_score, multiple_choice_score_full, multiple_choice_score_one_wrong,
                     multiple_choice_score_two_wrong, multiple_choice_score_three_wrong, multiple_choice_score_empty,
                     fill_choice_score_full, fill_choice_score_wrong):
    scores = []

    num_student_answer_cols = len(student_answers_sorted.columns) - 5 # 從第 6 欄（索引為 5）開始是答案，所以減5
    if num_student_answer_cols != len(correct_answers): # 使用 correct_answers 的長度來判斷
        raise ValueError(f"學生的答案欄位數量 ({num_student_answer_cols}) 與答案數量 ({len(correct_answers)}) 不符，請檢查學生的作答資料或答案是否上傳正確。")

    for idx, row in student_answers_sorted.iterrows():
        score_single_choice = 0
        score_multiple_choice = 0
        score_fill_choice = 0
        add_subtract_score = 0

        question_start_col = 5 #學生答案從第5欄開始

        # 單選題
        single_choice_start_col = question_start_col
        score_single_choice = calculate_single_choice_score(
            row, correct_answers, single_choice_start_col, num_single_choice, single_choice_score
        )

        # 多選題
        multiple_choice_start_col = single_choice_start_col + num_single_choice
        score_multiple_choice = calculate_multiple_choice_score(
            row, correct_answers, multiple_choice_start_col, num_multiple_choice,
            multiple_choice_score_full, multiple_choice_score_one_wrong,
            multiple_choice_score_two_wrong, multiple_choice_score_three_wrong, multiple_choice_score_empty
        )

        # 選填題
        fill_choice_start_col = multiple_choice_start_col + num_multiple_choice
        fill_choice_question_start_index_in_correct_answers = num_single_choice + num_multiple_choice + 1
        current_fill_choice_question = 0
        for num_cells in fill_choice_num_cells:
            correct_answer_start_index = fill_choice_question_start_index_in_correct_answers + sum(fill_choice_num_cells[:current_fill_choice_question])
            fill_score = calculate_fill_choice_score(
                row, correct_answers, fill_choice_start_col, num_cells,
                fill_choice_score_full, fill_choice_score_wrong, correct_answer_start_index
            )
            score_fill_choice += fill_score
            fill_choice_start_col += num_cells
            current_fill_choice_question += 1


        # 加減分=0
        add_subtract_score = 0

        total_score = score_single_choice + score_multiple_choice + score_fill_choice + add_subtract_score

        email = row.iloc[1]
        class_name = row.iloc[2]
        seat_number = row.iloc[3]
        name = row.iloc[4]

        scores.append({
            '電子郵件地址': email,
            '班級': class_name,
            '座號': seat_number,
            '姓名': name,
            '選擇題成績': score_single_choice,
            '多選題成績': score_multiple_choice,
            '選填題成績': score_fill_choice,
            '加減分': add_subtract_score,
            '總分': total_score
        })

    return pd.DataFrame(scores)

# ====== 批改每個班級的作答檔案 ======
for student_file_name, student_file_content in uploaded_students.items():
    print(f"\n====== 開始批改檔案：{student_file_name} ======")
    try:
        student_answers = pd.read_excel(io.BytesIO(student_file_content))
        student_answers_sorted = student_answers.sort_values(by=['班級', '座號']) # 假設學生作答檔案有'班級'欄位
    except Exception as e:
        print(f"錯誤：讀取學生作答檔案 '{student_file_name}' 失敗：{e}")
        continue

    # ====== 執行計算 (針對當前班級的學生作答) ======
    try:
        scores_df = calculate_scores(
            student_answers_sorted, correct_answers, num_single_choice, num_multiple_choice, fill_choice_num_cells,
            single_choice_score, multiple_choice_score_full, multiple_choice_score_one_wrong,
            multiple_choice_score_two_wrong, multiple_choice_score_three_wrong, multiple_choice_score_empty,
            fill_choice_score_full, fill_choice_score_wrong
        )

        # ====== 檢查並插入空行 (針對當前班級，並在班級之間插入空行) ======
        final_scores_list = []
        expected_seat_number = 1
        current_class = None

        if not scores_df.empty:
            for _, row in scores_df.iterrows():
                student_class = row['班級']
                student_seat = row['座號']

                if student_class != current_class and current_class is not None:
                    final_scores_list.append(pd.Series([None] * len(scores_df.columns), index=scores_df.columns))
                    expected_seat_number = 1

                while student_seat > expected_seat_number:
                    final_scores_list.append(pd.Series([None] * len(scores_df.columns), index=scores_df.columns))
                    expected_seat_number += 1

                final_scores_list.append(row)
                expected_seat_number += 1
                current_class = student_class

            # 處理最後一個班級的空行 (如果需要)
            if current_class is not None:
                max_seat_by_class = student_answers.groupby('班級')['座號'].max()
                last_seat = final_scores_list[-1]['座號'] if final_scores_list and isinstance(final_scores_list[-1], pd.Series) else 0
                max_expected_seat = max_seat_by_class.get(current_class, 0)
                while last_seat < max_expected_seat:
                    final_scores_list.append(pd.Series([None] * len(scores_df.columns), index=scores_df.columns))
                    last_seat += 1

        final_scores_df = pd.DataFrame(final_scores_list)

        # ====== 儲存結果檔案 (每個班級一個檔案) ======
        output_file_name = f"{os.path.splitext(student_file_name)[0]}_result.xlsx"
        final_scores_df.to_excel(output_file_name, index=False)
        print(f"批改結果已儲存至檔案：{output_file_name}\n")

    except ValueError as ve:
        print(f"警告：檔案 '{student_file_name}' 的答案數量或學生欄位數不符，已跳過計算分數。錯誤訊息：{ve}")
        continue # 跳過這個檔案的後續處理

    except Exception as e:
        print(f"批改檔案 '{student_file_name}' 時發生未知錯誤：{e}")
        continue
print('感恩人美心善的瑞祥，讚嘆人美心善的瑞祥！！！\n'*3)

# 答案範本格式輸出

In [ ]:
# @title
import pandas as pd
answer_example = {
    '小考名稱': ['204'],
    '題型數量': ['2,3,3,1,3,2,3,4,4,4,2,3,5,3,2,2,2'],
    '配分': ['6,6,4,2,6'],
    '選擇1答案（可不寫）': ['A'],
    '選擇2答案（可不寫）': ['B'],
    '多選1答案（可不寫）': ['A,B'],
    '選填1答案（可不寫）': ['-'],
    '選填2答案（可不寫）': ['3'],
    '選填3答案（可不寫）': ['0'],
    '': ['1'],
    '': ['2'],
}
df = pd.DataFrame(answer_example)
df.to_excel('答案範本.xlsx', index=False)
print("答案範本已儲存為 '答案範本.xlsx'。")

# 清除檔案
（如果有需要的話）執行程式用以清除暫存空間的所有檔案

In [ ]:
# @title
# 清除料夾下的檔案
import os
import shutil

folder_path = '/content/'

try:
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
            print(f"已刪除檔案：{item_path}")
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
            print(f"已刪除資料夾：{item_path}")
    print(f"成功清空資料夾：{folder_path}")
except Exception as e:
    print(f"清空資料夾時發生錯誤：{e}")